In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
data_sales = pd.read_csv("C:/Users/kanav/OneDrive/Desktop/ML/AWCustomers.csv")
# print(data_sales)

selected_columns = [
    'CustomerID', 'City', 'StateProvinceName', 'CountryRegionName','BirthDate' ,'Education', 'Occupation',
    'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
    'TotalChildren', 'YearlyIncome'
]
df = data_sales[selected_columns]
print(df)
data_types={
    'CustomerID':'Discrete',
    'City':'Nominal',
    'StateProvinceName':'Nominal',
    'CountryRegionName':'Nominal',
    'BirthDate':'Discrete',
    'Education':'Ordinal',
    'Occupation':'Nominal',
    'Gender':'Nominal',
    'MaritalStatus':'Nominal',
    'HomeOwnerFlag':'Discrete',
    'NumberCarsOwned':'Discrete Ratio',
    'TotalChildren':'Discrete Ratio',
    'YearlyIncome':'Continous'
}
print(data_types)



# ---------- Part II ----------
df = df.fillna(method='ffill')

# Normalization
scaler = MinMaxScaler()
df['Age_norm'] = scaler.fit_transform(df[['BirthDate']])
df['Income_norm'] = scaler.fit_transform(df[['YearlyIncome']])

# Discretization
df['Age_disc'] = pd.cut(df['BirthDate'], bins=[0,25,40,60,100], labels=['Youth','YoungAdult','Adult','Senior'])

# Standardization
std_scaler = StandardScaler()
df['Age_std'] = std_scaler.fit_transform(df[['BirthDate']])
df['Income_std'] = std_scaler.fit_transform(df[['YearlyIncome']])

# One-hot encode categorical
df = pd.get_dummies(df, columns=['City','StateProvinceName','CountryRegionName','Education','Occupation','Gender','MaritalStatus'])

# ---------- Part III ----------

obj1 = df.iloc[0]
obj2 = df.iloc[1]

smc = np.sum(obj1==obj2)/len(obj1)
jaccard = np.sum((obj1 & obj2)==1)/np.sum((obj1 | obj2)==1)
cos_sim = cosine_similarity([obj1],[obj2])[0][0]

corr = df['NumberCarsOwned'].corr(df['YearlyIncome'])

print("SMC:", smc)
print("Jaccard:", jaccard)
print("Cosine:", cos_sim)
print("Correlation:", corr)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Kanav/Downloads/adultdata.csv'